Installing the libraries

In [4]:
!pip install statsbombpy
!pip install nose2
!nose2 -v --pretty-assert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.6/205.6 kB 4.6 MB/s eta 0:00:00

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


All libraries are imported here

In [5]:
import pandas as pd
import numpy as np
from statsbombpy import sb


In [6]:
dir(sb)

['DEFAULT_CREDS',
 'MAX_CONCURRENCY',
 'Pool',
 'Union',
 '_360_frames',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'api_client',
 'competition_events',
 'competition_frames',
 'competitions',
 'events',
 'filter_and_group_events',
 'frames',
 'lineups',
 'matches',
 'merge_events_and_frames',
 'partial',
 'pd',
 'player_match_stats',
 'player_season_stats',
 'public',
 'reduce_events',
 'team_season_stats']

All the competitions that are available publicly using statsbomb

In [7]:
df = sb.competitions() # Show all competitions

FWC = df[df['competition_name'] == 'FIFA World Cup']
FWC # See all the tournaments for the Fifa World Cup

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
18,43,106,International,FIFA World Cup,male,False,True,2022,2023-06-24T17:17:27.911026,2023-06-24T17:18:55.629111,2023-06-24T17:18:55.629111,2023-06-24T17:17:27.911026
19,43,3,International,FIFA World Cup,male,False,True,2018,2022-09-05T17:17:56.670896,2021-06-13T16:17:31.694,None,2022-09-05T17:17:56.670896


In [8]:
df.loc[:, "competition_name"]

0            Champions League
1            Champions League
2            Champions League
3            Champions League
4            Champions League
5            Champions League
6            Champions League
7            Champions League
8            Champions League
9            Champions League
10           Champions League
11           Champions League
12           Champions League
13           Champions League
14           Champions League
15    FA Women's Super League
16    FA Women's Super League
17    FA Women's Super League
18             FIFA World Cup
19             FIFA World Cup
20        Indian Super league
21                    La Liga
22                    La Liga
23                    La Liga
24                    La Liga
25                    La Liga
26                    La Liga
27                    La Liga
28                    La Liga
29                    La Liga
30                    La Liga
31                    La Liga
32                    La Liga
33        

In [9]:
df.columns

Index(['competition_id', 'season_id', 'country_name', 'competition_name',
       'competition_gender', 'competition_youth', 'competition_international',
       'season_name', 'match_updated', 'match_updated_360',
       'match_available_360', 'match_available'],
      dtype='object')

All the matches that happened in the World Cup 2022

In [10]:
WC22 = sb.matches(competition_id= 43, season_id = 106).head(3) # They are 64 rows in total because there were 64 games.
WC22

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3857256,2022-12-02,21:00:00.000,International - FIFA World Cup,2022,Serbia,Switzerland,2,3,available,...,2023-04-26T23:49:58.956186,3,Group Stage,Stadium 974,Fernando Andrés Rapallini,Dragan Stojković,Murat Yakin,1.1.0,2,2
1,3869151,2022-12-03,21:00:00.000,International - FIFA World Cup,2022,Argentina,Australia,2,1,available,...,2023-05-14T23:35:15.370647,4,Round of 16,Ahmad bin Ali Stadium,Szymon Marciniak,Lionel Sebastián Scaloni,Graham James Arnold,1.1.0,2,2
2,3857257,2022-11-30,17:00:00.000,International - FIFA World Cup,2022,Australia,Denmark,1,0,available,...,2023-06-20T11:04:37.638969,3,Group Stage,Al Janoub Stadium,Mustapha Ghorbal,Graham James Arnold,Kasper Hjulmand,1.1.0,2,2


In [11]:
ARG_AUS_events = sb.events(match_id=3869151) # ARG VS AUS
ARG_AUS_events.head(3)

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Argentina,779,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 442, 'lineup': [{'player': {'id'...",Australia,792,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Australia,792,00:00:00.000,Half Start,NaN


In [12]:
sb.events(match_id=3869151, split=True, flatten_attrs=False)["dribbles"].head(3)

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,position,location,duration,under_pressure,related_events,dribble,match_id,possession_team_id,team_id,player_id
0,99b8ff1b-6a2e-4760-80e0-23ea3faad7b4,542,1,00:10:39.234,10,39,Dribble,19,Argentina,From Throw In,...,Left Wing,"[37.4, 19.6]",0.0,True,"[538e28ba-619c-4dc2-9000-813213a7d2b2, 9002f77...","{'outcome': {'id': 9, 'name': 'Incomplete'}}",3869151,779,779,7006
1,535ff683-fb17-453c-a244-47bb68ed945a,899,1,00:18:00.854,18,0,Dribble,27,Argentina,Regular Play,...,Right Back,"[33.3, 75.3]",0.0,True,[3bbe48cd-14c3-4d28-a52b-70d9a3495e6f],"{'outcome': {'id': 9, 'name': 'Incomplete'}}",3869151,779,779,29201
2,abba290b-f012-4617-83a5-629d6884a7bf,1446,1,00:29:08.965,29,8,Dribble,45,Australia,Regular Play,...,Right Back,"[19.4, 72.3]",0.0,True,[a704f209-00a7-42f4-a4c2-ca05c38d5c69],"{'outcome': {'id': 8, 'name': 'Complete'}}",3869151,792,792,15957


In [13]:
sb.lineups(match_id=3869151)["Argentina"].head(3)

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,player_id,player_name,player_nickname,jersey_number,country,cards,positions
0,2995,Ángel Fabián Di María Hernández,Ángel Di María,11,Argentina,[],[]
1,3090,Nicolás Hernán Otamendi,Nicolás Otamendi,19,Argentina,[],"[{'position_id': 5, 'position': 'Left Center B..."
2,5503,Lionel Andrés Messi Cuccittini,Lionel Messi,10,Argentina,[],"[{'position_id': 23, 'position': 'Center Forwa..."


In [32]:
arg_vs_aus = sb.events(match_id=3857254, split=True, flatten_attrs=False)
print(arg_vs_aus)
# arg_vs_aus['location'].max() # Here we can know that they are using the pitch dimensions in yards.

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


defaultdict(<class 'list'>, {'starting_xis':                                      id  index  period     timestamp  minute  \
0  a5e967a8-9457-4d5f-90fe-a974bb17b7dd      1       1  00:00:00.000       0   
1  d6b4ab33-02cc-4f33-99be-d12b01ee7f28      2       1  00:00:00.000       0   

   second         type  possession possession_team  play_pattern     team  \
0       0  Starting XI           1         Denmark  Regular Play  Denmark   
1       0  Starting XI           1         Denmark  Regular Play  Tunisia   

   duration                                            tactics  match_id  \
0       0.0  {'formation': 352, 'lineup': [{'player': {'id'...   3857254   
1       0.0  {'formation': 343, 'lineup': [{'player': {'id'...   3857254   

   possession_team_id  team_id  
0                 776      776  
1                 776      777  , 'half_starts':                                      id  index  period     timestamp  minute  \
0  8e1ef5ce-2ee6-4b7f-b27c-3352b5627745      3       1  00

In [15]:
arg_vs_aus.keys()

dict_keys(['starting_xis', 'half_starts', 'passes', 'ball_receipts', 'carrys', 'duels', 'pressures', 'miscontrols', 'clearances', 'ball_recoverys', 'interceptions', 'blocks', 'foul_committeds', 'foul_wons', 'dribbles', 'dispossesseds', 'shots', 'goal_keepers', 'tactical_shifts', 'dribbled_pasts', 'injury_stoppages', 'half_ends', 'substitutions', 'referee_ball_drops', 'own_goal_fors', 'own_goal_againsts'])

In [16]:
sb.events(match_id=3857254, split=True, flatten_attrs=False)["dribbles"].head(3) # get all the events dribbles for  Argentia vs Australia

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,position,location,duration,under_pressure,related_events,dribble,match_id,possession_team_id,team_id,player_id
0,99b8ff1b-6a2e-4760-80e0-23ea3faad7b4,542,1,00:10:39.234,10,39,Dribble,19,Argentina,From Throw In,...,Left Wing,"[37.4, 19.6]",0.0,True,"[538e28ba-619c-4dc2-9000-813213a7d2b2, 9002f77...","{'outcome': {'id': 9, 'name': 'Incomplete'}}",3869151,779,779,7006
1,535ff683-fb17-453c-a244-47bb68ed945a,899,1,00:18:00.854,18,0,Dribble,27,Argentina,Regular Play,...,Right Back,"[33.3, 75.3]",0.0,True,[3bbe48cd-14c3-4d28-a52b-70d9a3495e6f],"{'outcome': {'id': 9, 'name': 'Incomplete'}}",3869151,779,779,29201
2,abba290b-f012-4617-83a5-629d6884a7bf,1446,1,00:29:08.965,29,8,Dribble,45,Australia,Regular Play,...,Right Back,"[19.4, 72.3]",0.0,True,[a704f209-00a7-42f4-a4c2-ca05c38d5c69],"{'outcome': {'id': 8, 'name': 'Complete'}}",3869151,792,792,15957


In [17]:
# Combining all the events of the Fifa World Cup 2022
events = sb.competition_events(
    country="International",
    division= "FIFA World Cup",
    season="2022",
    gender="male"
)

WC_events22 = sb.competition_events(
    country="International",
    division= "FIFA World Cup",
    season="2022",
    split=True
)
# WC_events22['dribbles'] # Get all the dribbles that happened in the world cup

# grouped_events.drop('id', axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [18]:
# Get the list of column names or keys
key_names = WC_events22.keys()

# Print the column names
for key in key_names:
    print(key)

starting_xis
half_starts
passes
ball_receipts
carrys
clearances
ball_recoverys
shots
blocks
goal_keepers
miscontrols
pressures
duels
interceptions
foul_committeds
foul_wons
dispossesseds
dribbles
dribbled_pasts
errors
injury_stoppages
half_ends
substitutions
bad_behaviours
referee_ball_drops
tactical_shifts
player_offs
player_ons
shields
own_goal_fors
own_goal_againsts
50/50s
offsides


In [19]:
WC_events22['shots'].columns # Get all the dribbles that happened in the world cup

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'team', 'player',
       'position', 'location', 'duration', 'related_events', 'match_id',
       'shot_statsbomb_xg', 'shot_end_location', 'shot_technique',
       'shot_body_part', 'shot_type', 'shot_outcome', 'shot_first_time',
       'shot_freeze_frame', 'possession_team_id', 'team_id', 'player_id',
       'shot_key_pass_id', 'shot_one_on_one', 'shot_deflected',
       'under_pressure', 'shot_open_goal', 'shot_aerial_won', 'out',
       'shot_follows_dribble', 'shot_saved_to_post', 'shot_saved_off_target',
       'shot_redirect'],
      dtype='object')

get expected goals

In [20]:
WC_events22['clearances'].columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'team', 'player',
       'position', 'location', 'duration', 'under_pressure', 'related_events',
       'match_id', 'clearance_right_foot', 'clearance_body_part',
       'possession_team_id', 'team_id', 'player_id', 'clearance_left_foot',
       'clearance_aerial_won', 'clearance_head', 'out', 'off_camera',
       'clearance_other'],
      dtype='object')

In [21]:
xg_df = WC_events22['shots']
xG = xg_df.loc[:, 'player':'shot_statsbomb_xg']
xG

,player,position,location,duration,related_events,match_id,shot_statsbomb_xg
0,Granit Xhaka,Left Defensive Midfield,"[96.0, 38.8]",0.380264,"[23ce15e2-05d8-4adf-87de-90b82585c085, b27d5a6...",3857256,0.036566
1,Breel-Donald Embolo,Center Forward,"[113.1, 40.7]",0.175803,[ca8f5082-211d-4c5e-9827-b61cf7fa7ca2],3857256,0.353289
2,Granit Xhaka,Left Defensive Midfield,"[103.8, 41.9]",0.495575,[03e8c43d-a1ee-4477-a6cd-3761b9379837],3857256,0.069527
3,Nikola Milenković,Right Center Back,"[112.2, 36.8]",0.848375,[42dca3d5-9c35-4da4-8d13-0ae643b4d9c7],3857256,0.081609
4,Andrija Živković,Right Wing Back,"[97.8, 51.5]",0.934407,[66f1d9e3-640c-493d-80e6-33a247444389],3857256,0.030002
...,...,...,...,...,...,...,...
1489,Andreas Evald Cornelius,Center Forward,"[118.2, 43.9]",0.058813,[b2cf8522-fd3d-44ae-b3d2-6a2c2d40ce7a],3857254,0.568312
1490,Mathias Jensen,Left Defensive Midfield,"[102.5, 24.2]",0.995427,[100b467e-ced3-4e97-ab89-3656d3ab04c5],3857254,0.028724
1491,Andreas Evald Cornelius,Center Forward,"[114.4, 36.8]",1.239140,"[a4edfab9-529b-4f34-b150-bf665f559ce5, f1dcd43...",3857254,0.145394
1492,Joakim Mæhle,Left Back,"[95.0, 30.2]",0.164703,"[2da1a8a2-7c43-4ffa-91da-a4ed3d276e8c, 8c6aa51...",3857254,0.028684


In [22]:
# WC_events22[ WC_events22['interceptions']
WC_events22['passes']

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,pass_technique,pass_cut_back,pass_goal_assist,pass_through_ball,pass_miscommunication,counterpress,pass_no_touch,out,pass_straight,pass_inswinging
0,4acb4fd2-f46a-4d73-993c-e06597873924,5,1,00:00:00.521,0,0,Pass,2,Switzerland,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f21fda3d-cb2c-4608-af2d-201d1ad8101c,8,1,00:00:03.385,0,3,Pass,2,Switzerland,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57654ce3-e311-49c4-9ba7-4fab2ec66f71,11,1,00:00:06.697,0,6,Pass,2,Switzerland,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b4fc9de7-9029-4f18-bd6c-3d2a3c5e276a,13,1,00:00:08.484,0,8,Pass,2,Switzerland,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d89e79ae-d538-44c2-9a5f-f979984ef67a,16,1,00:00:15.905,0,15,Pass,2,Switzerland,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68509,e367ea9e-7288-4108-9c62-0c5c97df8727,3666,2,00:50:43.216,95,43,Pass,198,Denmark,From Throw In,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68510,1fcc3199-c5f7-4d35-a7b2-e41f6062ba93,3672,2,00:51:16.121,96,16,Pass,199,Denmark,From Corner,...,Straight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
68511,57e30122-783c-4e69-9657-6ad0697550c1,3677,2,00:51:35.423,96,35,Pass,200,Tunisia,From Keeper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68512,ba3fb28b-c572-4935-8548-e2adb22d8fb7,3680,2,00:51:57.342,96,57,Pass,201,Denmark,From Free Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
M: int = 12
N: int = 16

# 120, 80 yards

def get_cell_indexes(x, y, l, w):
    # Normalize x and y to the grid
    x = (x / field_length) * l
    y = (y / field_width) * w

    # Convert to integers and clip
    x = x.astype(int).clip(0, l - 1)
    y = y.astype(int).clip(0, w - 1)

    return x, y

def get_flat_indexes(x, y, l, w):
    # Get cell indexes
    x, y = get_cell_indexes(x, y, l, w)

    # Flatten to 1D indexes
    index = y * l + x

    return index

def count_actions(x, y, l, w):
    # Remove invalid points
    valid_points = (~x.isnull()) & (~y.isnull())
    x = x[valid_points]
    y = y[valid_points]

    # Get indexes and count
    index = get_flat_indexes(x, y, l, w)
    counts = index.value_counts(sort=False)

    # Create a zero matrix and place counts
    grid = np.zeros(w * l, dtype=int)
    grid[counts.index] = counts

    # Reshape to grid
    grid = grid.reshape((w, l))

    return grid

def safe_divide(a, b):
    # Perform safe division
    return np.divide(a, b, out=np.zeros_like(a, dtype=float), where=b != 0)


#https://github.com/ML-KULeuven/socceraction/blob/master/socceraction/xthreat.py


Creating our class xT to predict on

In [24]:
def shot_prob(player, start_loc, end_loc): # Finding the shooting probility at specific location Sxy

  pass

def mov_prob(player, start_loc, end_loc): # Finding the
  pass


# def

In [25]:
# class xT:
#   def __int__(self, x):
#     self.x = x

#   def get_x(self):
#     return self.x

#   def set_x(self, new_x):
#     self.x = new_x

In [63]:
WC_events22['carrys'].columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'team', 'player',
       'position', 'location', 'duration', 'related_events', 'match_id',
       'carry_end_location', 'possession_team_id', 'team_id', 'player_id',
       'under_pressure'],
      dtype='object')

In [27]:
Final_df = pd.DataFrame()
type(Final_df)

pandas.core.frame.DataFrame

In [71]:
# Merge the DataFrames based on 'match_id', 'index', and 'location' columns
# Now, create a new DataFrame by taking the 'column_name' column from df1
WC_events22_passes = WC_events22['passes'][['match_id','index', 'team', 'player' ,'timestamp','minute', 'second',  'location', 'type' ]].copy()
WC_events22_carrys = WC_events22['carrys'][['match_id','index','team', 'player' ,'location', 'timestamp', 'minute', 'second', 'type', 'carry_end_location']].copy()
WC_events22_dribbles = WC_events22['dribbles'][['match_id','index', 'team', 'player' ,'timestamp','minute', 'second',  'location', 'type' ]].copy()
WC_events22_shot = WC_events22['shots'][['match_id','index', 'team', 'player' ,'timestamp','minute', 'second',  'location', 'type']].copy()

# Merge the DataFrames based on 'index' and 'location' columns
# Combine the DataFrames by stacking them vertically (top on top)
final_df = pd.concat([WC_events22_passes, WC_events22_carrys, WC_events22_dribbles, WC_events22_shot

                      ], axis=0)
# The most important events that we have is shoot, pass, dribble, carry.

# Sort the final DataFrame by 'index' column
final_df.sort_values(by=['match_id','index'], inplace=True)

# Reset the index of the DataFrame
final_df.reset_index(drop=True, inplace=True)

final_df


,match_id,index,team,player,timestamp,minute,second,location,type,carry_end_location
0,3857254,5,Denmark,Kasper Dolberg,00:00:00.361,0,0,"[60.0, 40.0]",Pass,NaN
1,3857254,7,Denmark,Christian Dannemann Eriksen,00:00:00.580,0,0,"[58.0, 36.6]",Carry,"[59.3, 40.7]"
2,3857254,8,Denmark,Christian Dannemann Eriksen,00:00:01.560,0,1,"[59.3, 40.7]",Pass,NaN
3,3857254,10,Denmark,Andreas Skov Olsen,00:00:02.816,0,2,"[74.8, 52.3]",Carry,"[77.4, 54.9]"
4,3857254,12,Denmark,Andreas Skov Olsen,00:00:03.793,0,3,"[77.4, 54.9]",Pass,NaN
...,...,...,...,...,...,...,...,...,...,...
125560,3869685,4395,Argentina,Paulo Bruno Exequiel Dybala,00:02:27.184,122,27,"[108.1, 40.1]",Shot,NaN
125561,3869685,4397,France,Aurélien Djani Tchouaméni,00:03:29.289,123,29,"[108.1, 40.1]",Shot,NaN
125562,3869685,4399,Argentina,Leandro Daniel Paredes,00:04:16.722,124,16,"[108.1, 40.1]",Shot,NaN
125563,3869685,4402,France,Randal Kolo Muani,00:05:09.782,125,9,"[108.1, 40.1]",Shot,NaN


In [ ]:
# WC_events22_ball_receipts = WC_events22['ball_receipts'][['match_id','index', 'location' ]].copy()
# WC_events22_clearances = WC_events22['clearances'][['match_id','index', 'location' ]].copy()
# WC_events22_ball_recoverys = WC_events22['ball_recoverys'][['match_id','index', 'location' ]].copy()
# WC_events22_blocks = WC_events22['blocks'][['match_id','index', 'location' ]].copy()
# WC_events22_goal_keepers = WC_events22['goal_keepers'][['match_id','index', 'location' ]].copy()
# WC_events22_miscontrols = WC_events22['miscontrols'][['match_id','index', 'location' ]].copy()
# WC_events22_pressures = WC_events22['pressures'][['match_id','index', 'location' ]].copy()
# WC_events22_duels = WC_events22['duels'][['match_id','index', 'location' ]].copy()
# WC_events22_interceptions =  WC_events22['interceptions'][['match_id','index', 'location' ]].copy()
# WC_events22_foul_committeds = WC_events22['foul_committeds'][['match_id','index', 'location' ]].copy()
# WC_events22_foul_wons = WC_events22['foul_wons'][['match_id','index', 'location' ]].copy()
# WC_events22_dispossesseds = WC_events22['dispossesseds'][['match_id','index', 'location' ]].copy()
# WC_events22_dribbled_pasts = WC_events22['dribbled_pasts'][['match_id','index', 'location' ]].copy()
# WC_events22_errors =  WC_events22['errors'][['match_id','index', 'location' ]].copy()
# WC_events22_referee_ball_drops = WC_events22['referee_ball_drops'][['match_id','index', 'location' ]].copy()
# WC_events22_shields = WC_events22['shields'][['match_id','index', 'location' ]].copy()
# WC_events22_own_goal_fors = WC_events22['own_goal_fors'][['match_id','index', 'location' ]].copy()
# WC_events22_own_goal_againsts = WC_events22['own_goal_againsts'][['match_id','index', 'location' ]].copy()
# WC_events22_50_50s = WC_events22['50/50s'][['match_id','index', 'location' ]].copy()
# WC_events22_offsides = WC_events22['offsides'][['match_id','index', 'location' ]].copy()



 # WC_events22_ball_receipts, WC_events22_clearances, WC_events22_ball_recoverys,
                      # WC_events22_blocks, WC_events22_goal_keepers, WC_events22_miscontrols, WC_events22_pressures,
                      # WC_events22_duels, WC_events22_interceptions, WC_events22_foul_committeds, WC_events22_foul_wons,
                      # WC_events22_dispossesseds, WC_events22_dribbled_pasts, WC_events22_referee_ball_drops,
                      # WC_events22_shields, WC_events22_own_goal_fors, WC_events22_own_goal_againsts, WC_events22_50_50s,
                      # WC_events22_offsides

In [57]:
# 3857254, 20


# Function to get the event details based on match_id and index
def get_event_by_match_and_index(match_id, event_index):
    # Get the events for the specific match
    events = sb.events(match_id=match_id)

    # Check if the event_index exists in the DataFrame
    if event_index in events.index:
        event_details = events.loc[event_index] # if we want a general output
        event_details = events.loc[event_index, ['type', 'location', 'player']] # if we want specific columns

        return event_details
    else:
        return None  # Event with given index not found in the match

# Example usage

# Replace these values with the desired match_id and event_index
match_id = 3857254  # Replace with your match_id
event_index = 20  # Replace with the desired event index

# Get the event details for the given match_id and event_index
event_details = get_event_by_match_and_index(match_id, event_index)

if event_details is not None:
    print("Event found:")
    print(event_details)
else:
    print("Event not found. Please check your match_id and event_index.")


/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


Event found:
type                  Pass
location      [21.8, 60.2]
player      Yassine Meriah
Name: 20, dtype: object
